In [12]:
# coding: utf-8
import math
import time
import csv
import pandas as pd


class user_bought_history(object):

    def __init__(self):
        self.user_bought_history = []
        #	user_id			item_id			creat_at
        self.item_catogory = {}
        # key:item_id		value:catogory value
        self.result = {}
        # key: item_id 	value: frequency
        self.fashion_catogory_list = []
        # catogory
        self.have_init = False

    def init_cal_file(self):
        # read file
        with open('user_bought_history.txt', 'r')as f:
            items = f.readlines()

            for line in items:
                item = line.split()
                self.user_bought_history.append(item)

        with open('dim_items.txt', 'r')as f:
            items = f.readlines()

            for line in items:
                item = line.split()
                self.item_catogory[item[0]] = item[1]

        with open('fashion_cat.txt', 'r')as f:
            items = f.readlines()

            for line in items:
                item = line.split()
                self.fashion_catogory_list.append(item)

    def id_get_col(self, item_id):
        if not self.have_init:
            print("initialzing……")
            self.init_cal_file()
            self.have_init = True
        cat = self.item_catogory[item_id]
        return cat

    def cal(self, item_id):
        self.result = {}
        users_times = []

        # 将与商品相同的购买记录载入user_times表中
        for history in self.user_bought_history:
            if item_id == history[1]:
                users_times.append(history)

        # 找出同一买家在同一天购买的商品，若商品不属于同一类，则记录商品的购买频次
        cat_A = self.item_catogory[item_id]
        for user_time in users_times:
            for history in self.user_bought_history:
                if user_time[0] == history[0] and user_time[2] == history[2] and user_time[1] != history[1]:
                    cat_B = self.item_catogory[history[1]]
                    if cat_A != cat_B:
                        if history[1] in self.result:
                            # cal the f_pm: the frequency that a,b hanppend toghter
                            self.result[history[1]] += 1
                        else:
                            self.result[history[1]] = 1

        for key in self.result:
            cat_B = self.item_catogory[key]
            # 计算类与类之间的相似度
            f_cm = 0
            for item in self.fashion_catogory_list:
                if cat_A in item and cat_B in item:
                    f_cm += 1

            # 计算总相似度
            f_pm = self.result[key]
            self.result[key] = f_pm * math.log(1 + f_cm)
        # print("key:{0}, f_cm:{1}, f_pm:{2},result:{3}".format(key,f_cm,f_pm,self.result[key]))
        print("result.items(): ")
        print(self.result.items())
        self.result = sorted(self.result.items(), key=lambda item: item[1], reverse=True)
        # print(self.result)
        return self.result

    def single_sim(self, term_id):
        if not self.have_init:
            print("initialzing……")
            self.init_cal_file()
            self.have_init = True
            print("finishing initialzing……")
        sim = s.cal(term_id)
        sim = dict(sim)
        if sim:
            sim = self.dict_normalize(sim)
        print("sim: ")
        print(sim)
        return sim

    def dict_normalize(self, sim_dict):
        # 将相似度区间压缩到[0,1]
        max_value = max(sim_dict.values())
        min_value = min(sim_dict.values())
        if max_value != min_value:
            for key in sim_dict:
                sim_dict[key] = (sim_dict[key] - min_value) / (max_value - min_value)
        return sim_dict
    
  

if __name__ == '__main__':
    s = user_bought_history()
    result = []
    model_save_times = 0

    time1 = time.time()
    # 单个商品ID相似度检索，后期改成对所有商品ID相似度进行检索
    with open('model1_result_file.csv', 'wb') as f:
        id = input("请输入你想要寻找为此搭配的商品的id：")
        sim = s.single_sim(str(id))
        # 序列化数据，缺少此步写不进文档
        df = pd.DataFrame(sim,index=[str(id)])
        print(df)

    time2 = time.time()
    time_elapsed = time2 - time1
    print('耗时： {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))


请输入你想要寻找为此搭配的商品的id： 29


initialzing……
finishing initialzing……
result.items(): 
dict_items([('2485576', 5.993961427306569), ('476354', 6.887552571664617), ('1423178', 2.5649493574615367), ('2443037', 2.5649493574615367), ('1315712', 6.297109319933935), ('3138268', 5.327876168789581), ('140846', 2.772588722239781)])
sim: 
{'476354': 1.0, '1315712': 0.8634056325617346, '2485576': 0.7932747698373255, '3138268': 0.6391812235390243, '140846': 0.048035721644769325, '1423178': 0.0, '2443037': 0.0}
    476354   1315712   2485576   3138268    140846  1423178  2443037
29     1.0  0.863406  0.793275  0.639181  0.048036      0.0      0.0
耗时： 1m 8s


In [11]:
type(sim)

dict